# Gene Regulatory Network Analysis: Network Construction

This notebook demonstrates the construction and analysis of Gene Regulatory Networks using GRNBoost2.

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.grn_construction import GRNBuilder, NetworkAnalyzer
from src.visualization import NetworkVisualizer

## 1. Load Processed Data

In [ ]:
# Load expression data
tissue = "root"  # Change this for different tissues
expression_path = f"data/processed/{tissue}_processed.h5ad"
adata = sc.read_h5ad(expression_path)
expression_df = adata.to_df()

# Load TF list
tf_df = pd.read_csv("data/tf_list.csv")
tf_names = tf_df['GeneID'].unique().tolist()

print(f"Expression data shape: {expression_df.shape}")
print(f"Number of TFs: {len(tf_names)}")

## 2. Build Gene Regulatory Network

In [ ]:
# Initialize GRN builder
grn_builder = GRNBuilder(n_estimators=500)

# Set random seeds for multiple runs
seeds = [i * 111 for i in range(5)]

# Build GRN
grn_df = grn_builder.build_grn(
    expression_df=expression_df,
    tf_names=tf_names,
    seeds=seeds,
    top_edges_fraction=0.1
)

print(f"GRN edges: {len(grn_df)}")

## 3. Analyze Network Properties

In [ ]:
# Initialize network analyzer
analyzer = NetworkAnalyzer()

# Calculate network statistics
network_stats = analyzer.calculate_network_stats(grn_df)
print("\nNetwork Statistics:")
for stat, value in network_stats.items():
    print(f"{stat}: {value}")

In [ ]:
# Identify hub regulators
hub_regulators = analyzer.identify_hub_regulators(grn_df, min_targets=5)
print("\nTop 10 Hub Regulators:")
display(hub_regulators.head(10))

## 4. Analyze Expression-Regulation Relationships

In [ ]:
# Plot correlation vs importance
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=grn_df,
    x='PearsonR',
    y='InputImportance',
    alpha=0.5
)
plt.title('Expression Correlation vs. Regulatory Importance')
plt.xlabel('Pearson Correlation')
plt.ylabel('Regulatory Importance')
plt.show()

## 5. Network Visualization

In [ ]:
# Initialize network visualizer
visualizer = NetworkVisualizer()

# Plot full network
visualizer.plot_network(
    grn_df,
    min_appearance=0.8,
    min_edges=1,
    figsize=(20, 20)
)
plt.title(f"Gene Regulatory Network - {tissue.capitalize()} Tissue")
plt.show()

In [ ]:
# Plot subnetwork for top hub regulator
top_hub = hub_regulators.iloc[0]['TF']
visualizer.plot_network(
    grn_df,
    target_gene=top_hub,
    min_appearance=0.8,
    figsize=(15, 15)
)
plt.title(f"Regulatory Network for {top_hub}")
plt.show()

## 6. Save Results for Downstream Analysis

In [ ]:
# Save network data
output_dir = f"results/grn/{tissue}"
os.makedirs(output_dir, exist_ok=True)

# Save GRN edges
grn_df.to_csv(f"{output_dir}/grn_edges.csv", index=False)

# Save network statistics
pd.DataFrame([network_stats]).to_csv(
    f"{output_dir}/network_stats.csv",
    index=False
)

# Save hub regulators
hub_regulators.to_csv(f"{output_dir}/hub_regulators.csv", index=False)

print(f"Results saved to {output_dir}")

## 7. Cross-Tissue Analysis (Optional)

In [ ]:
# Load GRNs from different tissues
tissues = ['root', 'leaf', 'seed', 'shoot']
tissue_grns = {}

for t in tissues:
    grn_path = f"results/grn/{t}/grn_edges.csv"
    if os.path.exists(grn_path):
        tissue_grns[t] = pd.read_csv(grn_path)

# Compare networks across tissues
if len(tissue_grns) > 1:
    visualizer.plot_tissue_comparison(tissue_grns)
    plt.show()